In [1]:
import pandas as pd
import numpy as np
import sys

# Parent folder MUST be added.
# Otherwise it will not find the funcions module.
sys.path.append('../')

from functions.ETL import load_dfs

In [2]:
games, reviews, items = load_dfs()

DataFrames succesfully loaded.


In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item_id       32133 non-null  int64 
 1   developer     32133 non-null  object
 2   app_name      32133 non-null  object
 3   genres        32133 non-null  object
 4   tags          32133 non-null  object
 5   specs         32133 non-null  object
 6   release_year  32133 non-null  int64 
 7   price         30756 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.0+ MB


In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58430 entries, 0 to 58429
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    58430 non-null  object 
 1   item_id    58430 non-null  int64  
 2   recommend  58430 non-null  bool   
 3   sentiment  58430 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 1.4+ MB


In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246352 entries, 0 to 3246351
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int64 
 2   playtime_forever  int64 
dtypes: int64(2), object(1)
memory usage: 74.3+ MB


## Endpoints

In [7]:
def PlayTimeGenre(genre:str):
    """Return year with the highest number of 
    hours played for the provided ``genre``"""

    # Merged dataframe with release years and playtime
    merged = (
        items[['item_id', 'playtime_forever']]
        .merge(games[['item_id', 'app_name', 'genres', 'release_year']], how='left')
    )

    # Mask to filter out genres
    mask = (
        merged['genres']
        .map(lambda genres: genre in genres, na_action='ignore')
        .fillna(False)
    )

    # Filtering using mask, grouping and aggregate sum
    year = (
        merged[mask]
        .groupby(by='release_year') # Group by year
        .sum(numeric_only=True) # Aggregated sum by year
        ['playtime_forever'] #Selecting playtime
        .idxmax() # Getting the index for the maximum 
                  # which is now the year after grouping
    )

    response = {f'Release year with highest playtime for "{genre}" genre': int(year)}
    
    return response

In [8]:
PlayTimeGenre('Action')

{'Release year with highest playtime for "Action" genre': 2012}

In [9]:
def UserForGenre(genre: str):
    """Returns the user with the most hours played
    given the ``genre``."""

    # Merged DataFrame with user id, playtime
    # And genres for each item they've played
    merged = (
        items
        .merge(games[['item_id', 'genres', 'release_year']], how='left')
    )

    # Mask for filtering
    mask = (merged['genres']
        .map(lambda genres: genre in genres, na_action='ignore')
        .fillna(False)
    )

    # Filtering genres out, grouping by user and 
    # aggregating sum for playtime
    user = (
        merged[mask][['user_id', 'playtime_forever']]
        .groupby('user_id')
        .sum()
        # Sorting in descending order
        .sort_values(by='playtime_forever', ascending=False)
    ).iloc[0].name # Name of first record which is the user's id

    # Sum of hours played per year in a list
    years_played = (
        merged[mask][['user_id', 'release_year', 'playtime_forever']]
        .groupby(['user_id', 'release_year'])
        .sum()
        .loc[user]
    ) # This is a list with indexes as years and
      # values as the sum of hours played

    # Creating the response
    response = {
        'User': user,
        'Playtime_year': {f'Year {idx}': years_played.loc[idx, 'playtime_forever'] for idx in years_played.index}
    }
    
    return response

In [10]:
UserForGenre('Action')

{'User': 'Sp3ctre',
 'Playtime_year': {'Year 1995.0': 217,
  'Year 1999.0': 44,
  'Year 2000.0': 70644,
  'Year 2001.0': 13,
  'Year 2002.0': 238,
  'Year 2003.0': 7673,
  'Year 2004.0': 127411,
  'Year 2005.0': 21339,
  'Year 2006.0': 896,
  'Year 2007.0': 112784,
  'Year 2008.0': 224,
  'Year 2009.0': 108326,
  'Year 2010.0': 78083,
  'Year 2011.0': 154896,
  'Year 2012.0': 378296,
  'Year 2013.0': 120461,
  'Year 2014.0': 130691,
  'Year 2015.0': 312511,
  'Year 2016.0': 31233,
  'Year 2017.0': 43327}}

In [11]:
def UsersRecommend(year: int):
    """Top 3 of most recommended games for the 
    given ``year``."""

    # Merged DataFrame already filtered by 
    # recommendations and positive or neutral
    # reviews
    merged = (
        reviews.loc[(reviews['recommend'] == True) & (reviews['sentiment'] > 0)]
        .merge(games[['item_id', 'app_name', 'release_year']], how='left', on='item_id')
    )

    # Filtering by the year given
    masked = merged[merged['release_year'] == year]

    # Getting titles of games
    titles = (
        masked[['app_name', 'sentiment']] # Selecting 
        .groupby('app_name').sum() # Grouping by app_name
        .sort_values(by='sentiment', ascending=False) # Sorting in descending order
    )[:3].index # Getting the top 3 indexes (now the names)

    response = {f'Top {i+1}':titles[i] for i in range(3)}
    return response

In [12]:
UsersRecommend(2010)

{'Top 1': "Sid Meier's Civilization® V",
 'Top 2': 'Mount & Blade: Warband',
 'Top 3': 'Fallout: New Vegas'}

In [13]:
def UsersWorstDeveloper(year: int):
    """Top 3 developers with the least 
    recommended games for the given year.
    
    The criteria for selecting the top 3 
    is simply counting bad reviews for 
    each developer in an already-filtered
    DataFrame containing only negative reviews."""

    # Merged DataFrame already filtered by 
    # recommendations and negative reviews
    merged = (
    reviews.loc[(reviews['recommend'] == False) & (reviews['sentiment'] == 0)]
    .merge(games[['item_id', 'release_year', 'developer']], how='left', on='item_id')
    )

    # Filtering by the year given
    masked = merged[merged['release_year'] == year]

    # Getting titles
    titles = masked['developer'].value_counts()[:3].index

    # Creating json response
    response = {f'Top worst dev {i+1}':titles[i] for i in range(3)}

    return response

In [14]:
UsersWorstDeveloper(2010)

{'Top worst dev 1': 'id Software',
 'Top worst dev 2': 'Avalanche Studios',
 'Top worst dev 3': 'Telltale Games'}

In [15]:
def sentiment_analysis(dev: str):
    """Returns a dictionary containing
    the count for each review category.
    
    Negative: sentiment = 0 
    Neutral: sentiment = 1
    Positive: sentiment = 2
    """
    # Merged DataFrame
    merged = (
        reviews
        .merge(games[['item_id', 'release_year', 'developer']], how='left', on='item_id')
    )

    # Filtering developers
    masked = merged[merged['developer'] == dev]

    # Labels to assign
    labels = ['Negative', 'Neutral', 'Positive']
    # Counting and not sorting as indexes need to be ordered
    # to match labels
    revs_count = masked.value_counts('sentiment', sort=False)

    # Building the response json  usind dict comprehension
    response = {
        dev: [f'{label} = {value}' for (label, value) in zip(labels, revs_count)]
        }
    return response


In [16]:
sentiment_analysis('Valve')

{'Valve': ['Negative = 1095', 'Neutral = 2482', 'Positive = 6005']}